# Sampling Distirbutions with the NHANES Dataset

This notebook shows how to create sampling dirstibutions with the NHANES dataset. However, note that sampling distributions are not usually created -- they are rather a theoretical concept.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
da = pd.read_csv("nhanes_2015_2016.csv")